## 1: Study  of ``still`` ``lifes`` and ``oscillators``:  [5 marks]

For the following shapes from the data folder:
- Loaf ("data/Loaf.txt")
- Pulsar ("data/Pulsar.txt")
- Pentadecathlon ("data/Pentadecathlon.txt")

Find the number of live cells and the $r_{max}$  as a function of time (number of steps).<br>

Can you determine the period of the above 3 shapes from these plots?<br>
Hint: One does not change at all, the other 2 return back to their original configuration after certain steps.

Note: $r_{max}$ can be defined as:<br>
- Given N live cells at positions  ($i$,$j$) 
- Calculate the population "barycenter" ($center_{i}$,$center_{j}$), where $center_{i} =\frac{\sum_{i}^{N} i }{N}$ and $center_{j} =\frac{\sum_{j}^{N} j }{N}$
- Define the distance : $r_{i,j}= \sqrt{(i-center_i)^2+(j-center_j)^2}$ 
- $r_{max}$ is the maximum of all the $r_{i,j}$ i.e the distance of the furthest cell from the "barycenter" of the population.
    

In [3]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from itertools import product
from scipy import signal
import sys
import time

Loaf = np.loadtxt('data/Loaf.txt')      #Import configurations
Pulsar = np.loadtxt('data/Pulsar.txt')
Penta = np.loadtxt('data/Pentadecathlon.txt')

def alive(array):        #Function to find number of alive cells in a configuration
    N=(array==1).sum()
    return N

def rmax(array,N):        #Function to find rmax of a configuration
    alive=np.stack(np.nonzero(array)).T
    barycenter=((sum(alive[:,0])/N),(sum(alive[:,1])/N))
    list_alive=[]
    for i in alive:
        rmax=np.sqrt(((i[0]-barycenter[0])**2)+((i[1]-barycenter[1])**2))
        list_alive.append(rmax)
    return max(list_alive,default=0)

list_loaf=[Loaf]        #Creating lists with initial configurations and the corresponding values of rmax and alive
list_pulsar=[Pulsar]
list_penta=[Penta]
list_rmax_loaf=[rmax(Loaf,alive(Loaf))]
list_alive_loaf=[alive(Loaf)]
list_rmax_pulsar=[rmax(Pulsar,alive(Pulsar))]
list_alive_pulsar=[alive(Pulsar)]
list_rmax_penta=[rmax(Penta,alive(Penta))]
list_alive_penta=[alive(Penta)]

def GOL2(array):                               #Function to evolve the configuration forward one tick in time
    dummy=np.zeros(array.shape, dtype=int)                       #Create array of zeros with the same shape as the initial configuration
    neighbours=sum(np.roll(np.roll(array,i,0),j,1) for i in (-1,0,1) for j in (-1,0,1) if (i!=0 or j!=0))  #np.roll creates an array of the sum of the adjacent
    for i in range(array.shape[0]):                                                                        #values of all cells
        for j in range(array.shape[1]):
            if neighbours[i,j]==3:           #Conditions for the game, note since the dummy array is zeros, the conditions involving zeros do not need to be tested
                dummy[i,j]=1
            elif neighbours[i,j]==2 and array[i,j]==1:
                dummy[i,j]=1
    return dummy                             #Return the evolved configuration

def evolve(List1,List2,List3,List11,List12,List21,List22,List31,List32,N):
    for i in range(1,N):                          #Function to evolve the array using number of ticks and include values of rmax and alive, store values in lists
        List1.append(GOL2(List1[i-1]))
        List2.append(GOL2(List2[i-1]))
        List3.append(GOL2(List3[i-1]))
        List11.append(rmax((List1[i-1]),alive(List1[i-1])))
        List12.append(alive(List1[i-1]))
        List21.append(rmax((List2[i-1]),alive(List2[i-1])))
        List22.append(alive(List2[i-1]))
        List31.append(rmax((List3[i-1]),alive(List3[i-1])))
        List32.append(alive(List3[i-1]))

evolve(list_loaf,list_pulsar,list_penta,list_rmax_loaf,list_alive_loaf,list_rmax_pulsar,list_alive_pulsar,list_rmax_penta,list_alive_penta,45) #Evolve the config

plt.plot(list_rmax_loaf,'r',label='Loaf')           #Plot the variance in rmax
plt.plot(list_rmax_pulsar,'g',label='Pulsar')
plt.plot(list_rmax_penta,'b',label='Pentadecathlon')
plt.legend(loc='best')
plt.title('Variance of Rmax with Time')
plt.xlabel('Time Steps')
plt.ylabel('Rmax')
plt.show()

plt.plot(list_alive_loaf,'r',label='Loaf')          #Plot the variance in loaf
plt.plot(list_alive_pulsar,'g',label='Pulsar')
plt.plot(list_alive_penta,'b',label='Pentadecathlon')
plt.legend(loc='best')
plt.title('Variance of Number of Alive Cells with Time')
plt.xlabel('Time Steps')
plt.ylabel('Number of Alive Cells')
plt.show()

print('The period of Loaf does not change; the period of Pulsar is 4 and the period of Pentadecathlon is 16.')

The period of Loaf does not change; the period of Pulsar is 4 and the period of Pentadecathlon is 16.


## 2a: Produce a random configuration:  [5 marks]

For a 40x40 universe create a random configuration where :

1. 5% of the 20x20 cells at its center are alive (i.e 20 random cells at the 20x20 center).
2. 10% of the 20x20 cells at the center are alive (i.e 40 random cells at the 20x20 center).
3. 20% of the 20x20 cells at the center are alive (i.e 80 random cells at the 20x20 center).

Evolve it for N =100 steps.
Plot the number of live cells and their $r_{max}$ as a function of time (number of steps). 



In [4]:
def pad_with(vector, pad_width, iaxis, kwargs):       #Function to define what value an array is padded by
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
    return vector

def Random(ndead):                 #Function to make a random 20x20 array with a set number of 1s and 0s, then pad it to make it 40x40
    N=400 
    array=np.array([0]*ndead + [1]*(N-ndead))
    random=np.random.permutation(array)
    random=random.reshape((20,20))
    b=np.pad(random, 20, pad_with)
    return b

array5=Random(380)       #Make random configurations with 5%,10%,20% alive cells at center
array10=Random(360)
array20=Random(320)

list_5=[array5]           #Lists of original configurations
list_10=[array10]
list_20=[array20]
list_rmax_5=[rmax(array5,alive(array5))]
list_alive_5=[alive(array5)]
list_rmax_10=[rmax(array10,alive(array10))]
list_alive_10=[alive(array10)]
list_rmax_20=[rmax(array20,alive(array20))]
list_alive_20=[alive(array20)]

evolve(list_5,list_10,list_20,list_rmax_5,list_alive_5,list_rmax_10,list_alive_10,list_rmax_20,list_alive_20,100)    #Evolve configurations 100 steps

plt.plot(list_rmax_5,'r',label='5% Alive')      #Plot graphs of rmax and number of alive cells w.r.t. time
plt.plot(list_rmax_10,'g',label='10% Alive')
plt.plot(list_rmax_20,'b',label='20% Alive')
plt.legend(loc='best')
plt.title('Variance of Rmax for the 40x40 Arrays')
plt.xlabel('Time Steps')
plt.ylabel('Rmax')
plt.show()

plt.plot(list_alive_5,'r',label='5% Alive')
plt.plot(list_alive_10,'g',label='10% Alive')
plt.plot(list_alive_20,'b',label='20% Alive')
plt.legend(loc='best')
plt.title('Variance of Number of Alive Cells for the 40x40 Arrays')
plt.xlabel('Time Steps')
plt.ylabel('Number of Alive Cells')
plt.show()

/ext/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in long_scalars


## 3: Study of Random configurations:  [7 marks]

For a 40x40 universe generate three ensembles of 1000 random configurations each for the following scenarios:

1. 5% of the 20x20 cells at the center are alive (i.e 20 random cells at the 20x20 center).
2. 10% of the 20x20 cells at the center are alive (i.e 40 random cells at the 20x20 center).
3. 20% of the 20x20 cells at the center are alive (i.e 80 random cells at the 20x20 center).

For each ensemble : Plot the mean and the RMS of the population magnitute (number of live cells) and of the population  $r_{max}$ for 
t=0,50,100,200 steps.



In [5]:
e5,e10,e20=[],[],[] #Lists to contain the ensembles

for i in range(0,1000):    #Add the random configurations to the lists to create the ensembles
    a5=Random(380)
    e5.append(a5)
    a10=Random(360)
    e10.append(a10)
    a20=Random(320)
    e20.append(a20)

rmax_e5,rmax_e10,rmax_e20,alive_e5,alive_10,alive_20=[],[],[],[],[],[]  #Calculate rmax and alive cells for all of the initial configurations in the ensembles

for i in range(len(e5)):
    rmax_e5=[rmax((e5[i]),alive(e5[i]))]
    rmax_e10=[rmax((e10[i]),alive(e10[i]))]
    rmax_e20=[rmax((e20[i]),alive(e20[i]))]
    alive_e5=[alive(e5[i])]
    alive_e10=[alive(e10[i])]
    alive_e20=[alive(e20[i])]

evolve(e5,e10,e20,rmax_e5,alive_e5,rmax_e10,alive_e10,rmax_e20,alive_e20,200)    #Evolve the ensembles 200 time steps

t=[0,50,100,200]       #Make lists for values of time and empty lists to add calculated mean and rms values to
mean_e5_rmax,mean_e10_rmax,mean_e20_rmax,mean_e5_alive,mean_e10_alive,mean_e20_alive=[],[],[],[],[],[]
rms_e5_rmax,rms_e10_rmax,rms_e20_rmax,rms_e5_alive,rms_e10_alive,rms_e20_alive=[],[],[],[],[],[]

def Mean(List,N):            #Function to calculate mean of values in a list
    Mean=((sum(List))/N)*5
    return Mean

def RMS(List,N):             #Function to calculate rms of values in a list
    square=[]
    for i in List:
        square.append(i**2)
    RMS=math.sqrt(((sum(square))/N)*5)
    return RMS

def append(List,NewList,NewList2):      #Function to calculate the mean and rms of list slices, sliced to correspond to the required time values
    x1=Mean(List[0:1000],1000)
    x2=Mean(List[0:50000],50000)
    x3=Mean(List[0:100000],100000)
    x4=Mean(List[0:200000],200000)
    y1=RMS(List[0:1000],1000)
    y2=RMS(List[0:50000],50000)
    y3=RMS(List[0:100000],100000)
    y4=RMS(List[0:200000],200000)
    NewList.append(x1)
    NewList.append(x2)
    NewList.append(x3)
    NewList.append(x4)
    NewList2.append(y1)
    NewList2.append(y2)
    NewList2.append(y3)
    NewList2.append(y4)

append(rmax_e5,mean_e5_rmax,rms_e5_rmax)       #Function to add mean and rms values to the required lists
append(rmax_e10,mean_e10_rmax,rms_e10_rmax)
append(rmax_e20,mean_e20_rmax,rms_e20_rmax)
append(alive_e5,mean_e5_alive,rms_e5_alive)
append(alive_e10,mean_e10_alive,rms_e10_alive)
append(alive_e20,mean_e20_alive,rms_e20_alive)

plt.plot(t,mean_e5_rmax,'r',label='5% Alive')      #Plot the graphs required
plt.plot(t,mean_e10_rmax,'g',label='10% Alive')
plt.plot(t,mean_e20_rmax,'b',label='20% Alive')
plt.title('Mean Rmax')
plt.xlabel('Time Steps')
plt.ylabel('Rmax')
plt.show()

plt.plot(t,mean_e5_alive,'r',label='5% Alive')
plt.plot(t,mean_e10_alive,'g',label='10% Alive')
plt.plot(t,mean_e20_alive,'b',label='20% Alive')
plt.title('Mean Alive')
plt.xlabel('Time Steps')
plt.ylabel('Number of Alive Cells')
plt.show()

plt.plot(t,rms_e5_rmax,'r',label='5% Alive')
plt.plot(t,rms_e10_rmax,'g',label='10% Alive')
plt.plot(t,rms_e20_rmax,'b',label='20% Alive')
plt.title('RMS Rmax')
plt.xlabel('Time Steps')
plt.ylabel('Rmax')
plt.show()

plt.plot(t,rms_e5_alive,'r',label='5% Alive')
plt.plot(t,rms_e10_alive,'g',label='10% Alive')
plt.plot(t,rms_e20_alive,'b',label='20% Alive')
plt.title('RMS Alive')
plt.xlabel('Time Steps')
plt.ylabel('Number of Alive Cells')
plt.show()

<font color='red'>**Jupyter Kernel terminated:**</font> This might be caused by running out of memory or hitting a bug in some library (e.g., forking too many processes, trying to access invalid memory, etc.). Consider restarting or upgrading your project or running the relevant code directly in a terminal to track down the cause, as [explained here](https://github.com/sagemathinc/cocalc/wiki/KernelTerminated).

## 4: Game of life animations :  [5 marks]


For the following patterns present in the data folder:
- Glider ("data/Glider.txt") produces one figure for each of the first 15 steps of its evolution
- GliderGun ("data/GliderGun.txt") produce an animation (~60 frames, one frame per time step)

<img align="bottom" src="GliderGun.gif" HEIGHT="400" WIDTH="400"> 

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import animation
plt.rcParams["animation.html"] = "html5"

Glider = np.loadtxt('data/Glider.txt')     #Load in the configurations
Gun = np.loadtxt('data/GliderGun.txt')

list_glider=[Glider]     #Create lists to store initial configurations
list_gun=[Gun]

for i in range(1,60):                  #Evolve configurations forward in time
    list_glider.append(GOL2(list_glider[i-1]))
    list_gun.append(GOL2(list_gun[i-1]))

def animate(List):       #Function to animate the list of configurations
    fig=plt.figure()
    ims=[]
    for i in range(len(List)):
        im=plt.imshow(List[i],cmap=plt.cm.binary,interpolation='nearest',animated=True)    #Turn arrays into images and add images to list
        ims.append([im])
    ani=animation.ArtistAnimation(fig,ims,interval=50,blit=True,repeat_delay=1000)
    return ani

ani1=animate(list_glider)   #Use functions to display the animations
display(ani1)
ani2=animate(list_gun)
display(ani2)

<font color='red'>**Jupyter Kernel terminated:**</font> This might be caused by running out of memory or hitting a bug in some library (e.g., forking too many processes, trying to access invalid memory, etc.). Consider restarting or upgrading your project or running the relevant code directly in a terminal to track down the cause, as [explained here](https://github.com/sagemathinc/cocalc/wiki/KernelTerminated).

## 5: HighLife [7 marks]

Conway's Life is classified as following the $\textbf{B}$3/$\textbf{S}$23 rule.

Specifically a cell:
- is $\textbf{B}$orn if it has exactly 3 neighbours
- $\textbf{S}$urvives if it has 2 or 3 living neighbours 
- it dies otherwise.

Another member in the family of "Life" cellular automata is ``HighLife`` that follow the $\textbf{B}$36/$\textbf{S}$23 rule.

A cell:
- is $\textbf{B}$orn if it has 3 or 6 neighbours
- $\textbf{S}$urvives if it has 2 or 3 living neighbours 
- it dies otherwise.


Following the HighLife rules and  employing the Replicator pattern present in the data folder ("data/Replicator.txt").

1. Evolve it for N=36
2. Produce an animation of the above 36 frames showing the system evolution
3. Plot the Number of cells versus time
4. Plot the  $r_{max}$ as a function of time
5. Can you see why it is called a "Replicator"?    

In [1]:
Replicator = np.loadtxt('data/Replicator.txt')   #Load in replicator array

def GOL3(array):          #Function identical to GOL2 except for one change in the conditions
    dummy=np.copy(array)
    h,w=np.shape(array)[0]-1,np.shape(array)[1]-1
    for i in range(1,h):
        for j in range(1,w):
            neighbours=array[i-1,j-1]+array[i,j-1]+array[i+1,j-1]+array[i-1,j]+array[i+1,j]+array[i-1,j+1]+array[i,j+1]+array[i+1,j+1]
            if neighbours not in [2,3]:
                dummy[i,j]=0
            elif (neighbours in [3,6] and array[i,j]==0):     #Because a cell becomes alive if it has 3 or 6 neighbours now, '!=3' changes to 'in [3,6]'
                dummy[i,j]=0
            else:
                dummy[i,j]=1
    return dummy

list_rep=[Replicator]                                 #Lists containing initial configurations of the array
list_rep_rmax=[rmax(Replicator,alive(Replicator))]
list_rep_alive=[alive(Replicator)]

for i in range(1,37):                            #Evolve the configuration 36 steps in time
    list_rep.append(GOL3(list_rep[i-1]))
    list_rep_rmax.append(rmax((list_rep[i-1]),alive(list_rep[i-1])))
    list_rep_alive.append(alive(list_rep[i-1]))

plt.plot(list_rep_rmax,'r',label='Replicator')      #Plot how rms and no. of alive cells vary with time
plt.legend()
plt.xlabel('Time Steps')
plt.ylabel('Rmax')
plt.title('Rmax for Replicator Configuration')
plt.show()

plt.plot(list_rep_alive,'r',label='Replicator')
plt.legend()
plt.xlabel('Time Steps')
plt.ylabel('Number of Alive Cells')
plt.title('Number of Alive Cells for Replicator Configuration')
plt.show()

NameError: name 'np' is not defined

In [0]:
ani3=animate(list_rep)     #Animate the replicator configuration
display(ani3)

print('Called a replicator because the configuration expands to fill the universe.')

## Previous GOL2 function before optimisation using np.roll

In [0]:
def GOLX(array):                               #Function to evolve the configuration forward one tick in time
    dummy=np.copy(array)                       #Create copy of configuration to evolve
    h,w=np.shape(array)[0]-1,np.shape(array)[1]-1    #Define height and width of the configuration with -1 to account for edges of configuration
    for i in range(1,h):
        for j in range(1,w):
            neighbours=array[i-1,j-1]+array[i,j-1]+array[i+1,j-1]+array[i-1,j]+array[i+1,j]+array[i-1,j+1]+array[i,j+1]+array[i+1,j+1]  #Nearest neighbours of a cell
            if neighbours not in [2,3]:           #Conditions for the game
                dummy[i,j]=0
            elif (neighbours!=3 and array[i,j]==0):
                dummy[i,j]=0
            else:
                dummy[i,j]=1
    return dummy     #Return the evolved configuration